## 1. Why Async?

- Normally in Python:

- Synchronous code → executes line by line.

- If one task waits (like time.sleep(5) or network request), everything else stops.
- With asyncio, you can run multiple tasks concurrently (overlap I/O waits like API calls, DB queries, file reads).
It’s NOT the same as multithreading or multiprocessing.

2.1 Coroutine
A coroutine is a special function declared with async def. It returns a coroutine object, not a value, and must be awaited.

Breakdown of what happens:
1. Event Loop Starts
asyncio.run(main()) creates and runs an event loop. This loop manages all asynchronous tasks.

2. Coroutines Created
Inside main(), three coroutine objects are created by calling say_hello(). These are not started yet; they're just objects.

3. asyncio.gather(...) Starts the Coroutines
asyncio.gather(...) schedules all three coroutines to run concurrently on the same event loop. It doesn't use threads or processes; it uses cooperative multitasking.

4. Execution Begins
- Each coroutine:
- Prints "Hello".
- Awaits asyncio.sleep(10), which tells the event loop: "Pause me for 10 seconds, but go ahead and run other tasks in the meantime."
- Because sleep is non-blocking, the event loop continues executing other coroutines while one is waiting.
- So all three coroutines:
- Print "Hello" almost instantly.
- Begin their 10-second non-blocking sleep in parallel.
- After approximately 10 seconds, they resume and print "World" one after another or nearly simultaneously (depending on how the event loop schedules them).

5. Return Values Collected
- Each coroutine returns the elapsed time (around 10 seconds).
- asyncio.gather(...) collects these values in a list.

In [7]:
import subprocess
import sys

# Run the file with the same Python interpreter
result = subprocess.run([sys.executable, "async_example1.py"], capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)



STDOUT:
 Hello
World
Execution times: ('Total Time', 10.01495885848999)

STDERR:
 


In [9]:
import subprocess
import sys

# Run the file with the same Python interpreter
result = subprocess.run([sys.executable, "async_example2.py"], capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)



STDOUT:
 Hello
Hello
Hello
World
World
World
Execution times: [10.009049415588379, 10.009049415588379, 10.009049415588379]

STDERR:
 


In [31]:
#Threads and async
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor

start =  time.time()
def blocking_task(name):
    print(f"{name} started (blocking)")
    time.sleep(10)
    print(f"{name} finished")
    return f"{name} done"

loop = asyncio.get_running_loop()
#run_in_executor() expects a regular (sync) function, not an async def.
with ThreadPoolExecutor() as pool:
    results = await asyncio.gather(
        loop.run_in_executor(pool, blocking_task, "Task-1"),
        loop.run_in_executor(pool, blocking_task, "Task-2"),
        loop.run_in_executor(pool, blocking_task, "Task-3"),
    )
print("Results:", results,time.time()-start)



Task-1 started (blocking)
Task-2 started (blocking)
Task-3 started (blocking)
Task-1 finished
Task-2 finished
Task-3 finished
Results: ['Task-1 done', 'Task-2 done', 'Task-3 done'] 10.040251016616821


In [29]:
import subprocess
import sys

# Run the file with the same Python interpreter
result = subprocess.run([sys.executable, "async_example3.py"], capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)



STDOUT:
 [Request Block 1] Blocking task started
[Request Block 2] Blocking task started
[Request Block 3] Blocking task started[Request 1] Hello

[Request 2] Hello
[Request 3] Hello
[Request 1] Blocking task finished
[Request 2] Blocking task finished
[Request 3] Blocking task finished
[Request 1] World
[Request 2] World
[Request 3] World
[Request 1] Results: ['[Request 1] Async Done', '[Request 1] Blocking Done']
[Request 2] Results: ['[Request 2] Async Done', '[Request 2] Blocking Done']
[Request 3] Results: ['[Request 3] Async Done', '[Request 3] Blocking Done']

STDERR:
 


In [34]:
import subprocess
import sys

# Run the file with the same Python interpreter
result = subprocess.run([sys.executable, "async_example4.py"], capture_output=True, text=True)

print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)



STDOUT:
 Task-1 started (in process)
Task-2 started (in process)
Task-3 started (in process)
Task-2 finished
Task-1 finished
Task-3 finished
Results: ['Task-1 done', 'Task-2 done', 'Task-3 done']

STDERR:
 
